In [ ]:
import pandas as pd
from pathlib import Path
import numpy as np
import sys

# Determine the project root
current_path = Path.cwd().resolve()
root = current_path.parents[0]

# find modelling path
mod_path = root / "src" / "visualizations"

# append path for custom function
sys.path.append(str(mod_path))

# load stationarity function
from stationarity_report import build_stationarity_html

In [ ]:
# aini dfs
data_path = root / "data"
var_path = data_path / "processed" / "variables"
aini_df = pd.DataFrame()
vars = ["binary", "w0", "w1", "w2"]

aini_dfs = {}

for var in vars:
    df = pd.read_csv(var_path / f"{var}_AINI_variables.csv")

    # ensure types and order
    df["date"] = pd.to_datetime(df["date"], errors="coerce")
    df["normalized_AINI"] = pd.to_numeric(df["normalized_AINI"], errors="coerce")
    df = df.sort_values("date").reset_index(drop=True)
    aini_dfs[var] = df

# target cols
target_cols_aini = ["EMA_02","EMA_08","normalized_AINI"]
aini_data_test = list()

# iterate through models and variables
for key, df in aini_dfs.items():
    for var in target_cols_aini:
        if var in df.columns:
            temp = pd.DataFrame({
                "Date": pd.to_datetime(df["date"]),
                "measure": f"{key}_{var}",
                "value": df[var].values
            })
            aini_data_test.append(temp)

# concatenate all partial dataframes
aini_data_test = pd.concat(aini_data_test, ignore_index=True)

aini_data_test

In [ ]:
# n articles
n_art = pd.read_csv(var_path / "n_articles.csv")
n_art["Date"] = pd.to_datetime(n_art["date"], errors="coerce")

# ensure types and order 
n_art["n_articles"] = pd.to_numeric(n_art["n_articles"], errors="coerce")
n_art = n_art.sort_values(["Date"]).reset_index(drop=True)

# log growth
n_art["log_count_growth"] = (
    n_art["n_articles"]
            .transform(lambda s: np.log(s).diff())
)

n_art_test = pd.DataFrame()
n_art_test["Date"] = n_art["Date"] 
n_art_test["measure"] = "log growth n articles"
n_art_test["value"] = n_art["log_count_growth"] 
n_art_test.to_csv(var_path / "n_articles_log.csv")

In [ ]:
# fin data
fin_path = data_path / "raw" / "financial" / "full_daily_2023_2025.csv"
fin_data = pd.read_csv(fin_path)

# ensure types and order 
fin_data["Date"] = pd.to_datetime(fin_data["Date"], errors="coerce")
fin_data["Adj Close"] = pd.to_numeric(fin_data["Adj Close"], errors="coerce")
fin_data = fin_data.sort_values(["Ticker", "Date"]).reset_index(drop=True)

# log returns per Ticker: Δ log(Adj Close)
fin_data["log_return"] = (
    fin_data.groupby("Ticker")["Adj Close"]
            .transform(lambda s: np.log(s).diff())
)

# target col
target_fin = ["log_return"]
fin_data

# ensure types and order 
fin_data["Date"] = pd.to_datetime(fin_data["Date"], errors="coerce")
fin_data["Adj Close"] = pd.to_numeric(fin_data["Adj Close"], errors="coerce")
fin_data = fin_data.sort_values(["Ticker", "Date"]).reset_index(drop=True)

# log returns per Ticker: Δ log(Adj Close)
fin_data["log_return"] = (
    fin_data.groupby("Ticker")["Adj Close"]
            .transform(lambda s: np.log(s).diff())
)

# target col
target_fin = ["log_return"]
fin_data_test = pd.DataFrame()

# subset 
fin_data_test["Date"] = fin_data["Date"] 
fin_data_test["measure"] = fin_data["Ticker"] 
fin_data_test["value"] = fin_data["log_return"] 

fin_data_test

In [ ]:
# vix data
vix_path = data_path / "processed" / "variables" / "log_growth_VIX.csv"
vix_data = pd.read_csv(vix_path)

# subset 
vix_data_test = pd.DataFrame()
vix_data_test["Date"] = pd.to_datetime(vix_data["date"])
vix_data_test["measure"] = "VIX"
vix_data_test["value"] = vix_data["log_growth_closed"]

vix_data_test

In [ ]:
# concat all 
test_all = pd.concat([vix_data_test,fin_data_test,aini_data_test,n_art_test])
test_all = test_all.rename({"measure":"Measure","value":"Value"},axis=1)
test_all

In [ ]:
combined = build_stationarity_html(
    df=test_all,  
    output_html= root / "reports/tables/stationarity_all_measures.html",
    save_combined_csv= root /"data/processed/variables/stationarity_all_measures.csv",
    save_combined_tex= root /"reports/tables/stationarity_all_measures.tex",
)
